In [1]:
import os
from azureml.core import Workspace, Dataset, Datastore
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.pipeline.wrapper import Module, dsl

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

aml_compute_target = "aml-compute"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("Found existing compute target: {}".format(aml_compute_target))
except:
    print("Creating new compute target: {}".format(aml_compute_target))

    provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                                min_nodes=1,
                                                                max_nodes=4)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

DesignerTest-EUS
DesignerTestRG
eastus
4faaaf21-663f-4391-96fd-47197c630979
Found existing compute target: aml-compute


In [ ]:
global_datastore = Datastore(
            ws,
            name="azureml_globaldatasets")
movie_ratings_data = Dataset.File.from_files(
            global_datastore.path('GenericCSV/Movie_Ratings')).as_named_input('Movie_Ratings')

In [2]:
copyfiles_module_func = Module.from_yaml(ws, yaml_file=os.path.join(
            'modules', 'copy_files', 'copy_files.spec.yaml'))

In [ ]:
copyfiles_module_func()

In [ ]:
@dsl.pipeline(
    name='copy_files', description='copy_files',
    default_compute_target='aml-compute')
def copy_files_pipeline():
    step1 = copyfiles_module_func(input_dir=movie_ratings_data)
    copyfiles_module_func(input_dir=step1)

In [ ]:
pipeline = copy_files_pipeline()

In [ ]:
pipeline.validate()

In [ ]:
run = pipeline.submit(
    experiment_name='copy_files_experiment'
)
run.wait_for_completion()
run